In [59]:
import numpy as np 
import json
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

In [10]:
CHATBOT_FILE = 'E:\BANGKIT2022\CapstoneProject\data_gathering\data_chatbot\intents.json'

# Read the data
f = open(CHATBOT_FILE)
load = json.load(f)

X_train = []
y_train = []
for i in load['intents']:
    for x in i['patterns']:
        X_train.append(x.lower())
        y_train.append(i['tag'])



In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
total_words = len(tokenizer.word_index) + 1

In [18]:
print(tokenizer.word_index)

{'selamat': 1, 'makanan': 2, 'halo': 3, 'siang': 4, 'pagi': 5, 'malam': 6, 'tolong': 7, 'rekomendasikan': 8, 'rekomendasi': 9, 'meal': 10, 'yang': 11, 'cocok': 12, 'saran': 13}


In [22]:
tokenizer.texts_to_sequences(X_train)

[[3], [1, 4], [1, 5], [1, 6], [7, 8, 2], [9, 10], [2, 11, 12], [13, 2]]

In [43]:
def n_gram_seqs(corpus, tokenizer, label):
	input_sequences = []
	new_label = []
	### START CODE HERE
	for index, line in enumerate(corpus):
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(0, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)
			new_label.append(label[index])
	### END CODE HERE
	 
	return input_sequences, new_label

In [44]:
first_example_sequence = n_gram_seqs([X_train[0]], tokenizer)

print("n_gram sequences for first example look like this:\n")
first_example_sequence

TypeError: n_gram_seqs() missing 1 required positional argument: 'label'

In [48]:
input_sequences, y_train = n_gram_seqs(X_train, tokenizer, y_train)

max_sequence_len = max([len(x) for x in input_sequences])

print(f"n_grams of input_sequences have length: {len(input_sequences)}")
print(f"maximum length of sequences is: {max_sequence_len}")

n_grams of input_sequences have length: 17
maximum length of sequences is: 3


In [49]:
# GRADED FUNCTION: pad_seqs
def pad_seqs(input_sequences, maxlen):
    ### START CODE HERE
    padded_sequences = pad_sequences(input_sequences, maxlen=maxlen)
    
    return padded_sequences
    ### END CODE HERE

In [51]:
input_sequences = pad_seqs(input_sequences, max_sequence_len)

print(f"padded corpus has shape: {input_sequences.shape}")
print(input_sequences, y_train)

padded corpus has shape: (17, 3)
[[ 0  0  3]
 [ 0  0  1]
 [ 0  1  4]
 [ 0  0  1]
 [ 0  1  5]
 [ 0  0  1]
 [ 0  1  6]
 [ 0  0  7]
 [ 0  7  8]
 [ 7  8  2]
 [ 0  0  9]
 [ 0  9 10]
 [ 0  0  2]
 [ 0  2 11]
 [ 2 11 12]
 [ 0  0 13]
 [ 0 13  2]] ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi', 'rekomendasi']


In [53]:
preprocessing.LabelEncoder()
label_encoder = preprocessing.LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [54]:
y_train_encoded

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [60]:
model = tf.keras.Sequential([
  layers.Dense(32, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(len(y_train_encoded), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=input_sequences,
          y=y_train_encoded,
          epochs=500)

Epoch 1/500
1/1 [==============================] - 5s 5s/step - loss: 3.9632 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 2.5034 - accuracy: 0.2353
Epoch 3/500
1/1 [==============================] - 0s 5ms/step - loss: 1.5687 - accuracy: 0.5882
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 1.1740 - accuracy: 0.5882
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0101 - accuracy: 0.5882
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 0.9152 - accuracy: 0.5882
Epoch 7/500
1/1 [==============================] - 0s 7ms/step - loss: 0.8301 - accuracy: 0.5882
Epoch 8/500
1/1 [==============================] - 0s 10ms/step - loss: 0.7535 - accuracy: 0.5882
Epoch 9/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.5882
Epoch 10/500
1/1 [==============================] - 0s 9ms/step - loss: 0.6314 - accuracy: 0.5882
Epoch 11/500
1/1 [=======

In [92]:
seed_text = 'selamat pagi'

token_list = tokenizer.texts_to_sequences([seed_text])[0]
	# Pad the sequences
token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')

predicted = model.predict(token_list, verbose=0)
	# Choose the next word based on the maximum probability
predicted = np.argmax(predicted, axis=-1).item()

In [93]:
label = ['greeting','rekomendasi']
print(predicted, label[predicted])

0 greeting


In [80]:
saved_model_path = "./saved_model/simple_chatbot_model/first_model.h5"
tokenizer_path = "./saved_model/simple_chatbot_model/tokenizer.json"
max_seq_path = "./saved_model/simple_chatbot_model/max_sequence_length.txt"
# YOUR CODE HERE
model.save(saved_model_path)
with open(tokenizer_path, 'w') as file:
    json.dump(tokenizer.to_json(), file)
    file.close()
with open(max_seq_path, 'w') as file:
    file.write(str(max_sequence_len))
    file.close()